## Compute lattice constant using the current potential.

In [1]:
from ase.build import bulk
from ase.db import connect
from ase.eos import calculate_eos

input = {}
with open('Input.txt', 'r') as file:
    for line in file:
        key, value = line.strip().split(' : ')
        try:
            #Convert to integer if possible
            input[key] = int(value)
        except ValueError:
            # If not possible, store as string
            input[key] = value

atoms = bulk(input['surface_atom'], input['lattice'])
if (input['calc_method'] == 'EMT'):
    from ase.calculators.emt import EMT
    atoms.calc = EMT()
elif (input['calc_method'] == 'LJ'):
    from ase.calculators.lj import LennardJones
    atoms.calc = LennardJones()
elif (input['calc_method'] == 'EAM'):
    from ase.calculators.eam import EAM
    atoms.calc = EAM()

eos = calculate_eos(atoms) #For now eos only seems to work with EMT, not LJ or EAM
v, e, B = eos.fit()  # find minimum
# Do one more calculation at the minimum and write to database:
atoms.cell *= (v / atoms.get_volume())**(1 / 3)
atoms.get_potential_energy()

-0.007036492048378307

## Adsorb one C atom using built-in BFGS.

### First prepare the supercell (so the atom adsorbate does not see its mirror image).

In [2]:
# Now prepare adsorption.
from ase.build import add_adsorbate, fcc111
from ase.visualize import view
ads = input['adsorbant_atom']
n_layers = input['number_of_layers']
a = atoms.cell[0, 1] * 2 # Equilibrium lattice constant.
atoms = fcc111(input['surface_atom'], (input['supercell_x_rep'], input['supercell_y_rep'], n_layers), a=a)
atoms.get_tags()

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1])

In [ ]:
view(atoms, viewer='x3d')

In [ ]:
atoms
atoms.get_positions()

## Add single atom adsorbate.

In [3]:
ads_height = float(input['adsorbant_init_h']) #modified initial conditions
add_adsorbate(atoms, ads, height=ads_height, position=input['lattice'])
#defined vacuum layer for no overlap between supercells NEEDS CONVERGENCE STUDY
atoms.center(vacuum = 10, axis = 2) 
atoms.get_tags()
atoms.get_positions()

array([[ 1.26919466,  0.73276988, 10.        ],
       [ 3.80758399,  0.73276988, 10.        ],
       [ 6.34597331,  0.73276988, 10.        ],
       [ 8.88436264,  0.73276988, 10.        ],
       [ 2.53838933,  2.93107952, 10.        ],
       [ 5.07677865,  2.93107952, 10.        ],
       [ 7.61516798,  2.93107952, 10.        ],
       [10.1535573 ,  2.93107952, 10.        ],
       [ 3.80758399,  5.12938916, 10.        ],
       [ 6.34597331,  5.12938916, 10.        ],
       [ 8.88436264,  5.12938916, 10.        ],
       [11.42275197,  5.12938916, 10.        ],
       [ 5.07677865,  7.3276988 , 10.        ],
       [ 7.61516798,  7.3276988 , 10.        ],
       [10.1535573 ,  7.3276988 , 10.        ],
       [12.69194663,  7.3276988 , 10.        ],
       [ 0.        ,  1.46553976, 12.07258621],
       [ 2.53838933,  1.46553976, 12.07258621],
       [ 5.07677865,  1.46553976, 12.07258621],
       [ 7.61516798,  1.46553976, 12.07258621],
       [ 1.26919466,  3.6638494 , 12.072

In [ ]:
view(atoms, viewer='x3d')


In [ ]:
atoms[-1]

In [4]:
# Constrain all atoms except the adsorbate:
from ase.constraints import FixAtoms
fixed = list(range(len(atoms) - 1))
atoms.constraints = [FixAtoms(indices=fixed)]

## Optimize adsorbate position usgin built-in BFGS from ASE.

In [5]:
from ase.optimize import BFGS
atoms.calc = EMT()
opt = BFGS(atoms, logfile=None)
opt.run(fmax=0.0001)

True

In [6]:
# Final adsorbate position.
### print(atoms[3].position)
print(atoms[-1].position) # modified to get adsorbate position

# Final energy.
print(atoms.get_potential_energy())

[ 1.26919466  0.73276988 15.5051332 ]
11.22491804732231


## Comparison with BoTorch

In [7]:
import numpy as np
import torch

In [8]:
# Do not allow our atom to go inside the surface. 
# Also restrict x-y to the unit cell size.
bulk_z_max = np.max(atoms[:-1].positions[:, 2]) #modified to account for changes in initial conditions + universal
print(bulk_z_max)
cell_x_min, cell_x_max = float(np.min(atoms.cell[:, 0])), float(np.max(atoms.cell[:, 0]))
cell_y_min, cell_y_max = float(np.min(atoms.cell[:, 1])), float(np.max(atoms.cell[:, 1]))
#z_adsorb_max = 3 * ads_height
z_adsorb_max = atoms[-1].position[-1] + 5 # modified to account for changes in initial conditions

14.145172411082733


## Call functions from separate file instead of defining in notebook

In [9]:
import pickle
with open('atoms.pkl', 'wb') as f:
    pickle.dump(atoms, f)

import Note_func

## Set up the optimization experiment in Ax.

In [10]:
from ax.service.ax_client import AxClient
from Note_func import gs
from ax.service.utils.instantiation import ObjectiveProperties
# Initialize the client - AxClient offers a convenient API to control the experiment
ax_client = AxClient(generation_strategy=gs)

ax_client.create_experiment(
    name="adsorption_experiment",
    parameters=[
        {
            "name": "x",
            "type": "range",
            "bounds": [float(cell_x_min), float(cell_x_max)],
        },
        {
            "name": "y",
            "type": "range",
            "bounds": [float(cell_y_min), float(cell_y_max)],
        },
        {
            "name": "z",
            "type": "range",
            "bounds": [float(bulk_z_max), float(z_adsorb_max)], #I made a modification here, switched both bounds.
        },
    ],
    objectives={"adsorption_energy": ObjectiveProperties(minimize=True)},
    # parameter_constraints=["x1 + x2 <= 2.0"],  # Optional.
    # outcome_constraints=["l2norm <= 1.25"],  # Optional.
)

[WARNING 03-09 17:14:54] ax.service.utils.with_db_settings_base: Ax currently requires a sqlalchemy version below 2.0. This will be addressed in a future release. Disabling SQL storage in Ax for now, if you would like to use SQL storage please install Ax with mysql extras via `pip install ax-platform[mysql]`.
[INFO 03-09 17:14:54] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
[INFO 03-09 17:14:54] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 03-09 17:14:54] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter y. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' o

## Run the BO loop.

In [11]:
from Note_func import evaluate
N_BO_steps = input['n_bo_steps']
for i in range(N_BO_steps):
    parameters, trial_index = ax_client.get_next_trial()
    # Local evaluation here can be replaced with deployment to external system.
    ax_client.complete_trial(trial_index=trial_index, raw_data=evaluate(parameters))

[INFO 03-09 17:14:56] ax.service.ax_client: Generated new trial 0 with parameters {'x': 6.383844, 'y': 4.526849, 'z': 17.334144} using model Sobol.
C:\Users\arnau\AppData\Roaming\Python\Python310\site-packages\ase\atoms.py:967: VisibleDeprecationWarning: Use get_global_number_of_atoms() instead
  warnings.warn('Use get_global_number_of_atoms() instead',
[INFO 03-09 17:14:56] ax.service.ax_client: Completed trial 0 with data: {'adsorption_energy': (14.085002, 0.0)}.
[INFO 03-09 17:14:56] ax.service.ax_client: Generated new trial 1 with parameters {'x': 2.526671, 'y': 2.704544, 'z': 16.63266} using model Sobol.
C:\Users\arnau\AppData\Roaming\Python\Python310\site-packages\ase\atoms.py:967: VisibleDeprecationWarning: Use get_global_number_of_atoms() instead
  warnings.warn('Use get_global_number_of_atoms() instead',
[INFO 03-09 17:14:56] ax.service.ax_client: Completed trial 1 with data: {'adsorption_energy': (13.116128, 0.0)}.
[INFO 03-09 17:14:56] ax.service.ax_client: Generated new tri

## (Optionnal : Load previous experiment result from JSON file)

In [ ]:
ax_client = AxClient.load_from_json_file(filepath='test_json.json')

## Display Results

In [ ]:
df = ax_client.get_trials_data_frame()
#save df as csv file
df.to_csv('trials_dataframe.csv', index=False)
ax_client.get_trials_data_frame()

## Plot Evolution of adsorption energy.

In [ ]:
from ax.utils.notebook.plotting import render
# from botorch.acquisition

render(ax_client.get_optimization_trace(objective_optimum=0.0)) #objective optimum should be the bare surface without additionnal atom

## Plot learned response surface.

In [ ]:
from ax.plot.contour import interact_contour
model = ax_client.generation_strategy.model
render(interact_contour(model=model, metric_name="adsorption_energy",
                       slice_values={'x': 1.263480218001716, 'y': 1.0, 'z': 3.01}))

In [ ]:
ax_client.get_best_parameters()
#Modify atoms to represent the best solution
params = ax_client.get_best_parameters()[:1][0]
atoms[-1].position[:] = params['x'],params['y'],params['z']
print(ax_client.get_best_parameters())

## Save Ax Client and experiment as JSON file

In [ ]:
ax_client.save_to_json_file(filepath= 'test_json.json')

## Visualize the resulting chemical system.

In [ ]:
from ase.visualize import view
view(atoms, viewer='x3d')

In [ ]:
import matplotlib.pyplot as plt
from ase.visualize.plot import plot_atoms

fig, ax = plt.subplots()

plot_atoms(atoms, ax, radii=0.5, rotation=('90x,45y,0z'))
#plot_atoms(atoms, ax, radii=0.5, rotation=('0x,0y,0z'))


fig.savefig("ase_slab.png")